# Lab 12

### oh its lab 12 alright lets gooooooo little robot

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

import time
import math
import csv
import re

import matplotlib as mpl
import matplotlib.pyplot as plt

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

# Start the plotter
START_PLOTTER()

from enum import IntEnum

class MODE_TYPE(IntEnum):
    SEEK_ANGLE = 0
    TAKE_READINGS = 1
    MOVE_FORWARD = 2
    BRAKE = 3
    BUG = 4
    
log_complete = False

class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    async def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        
        global tof
        global pose
        global CURR_LOG_ARRAY
        global log_complete
        
        tof = []
        pose = []
        
        print("starting reading")
        ble.send_command(CMD.SET_PID_GAINS,"ROTATION_RATE:|-0.01:|-0.001:|0.0")
        ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|0.01:|0.0002:|0.0")
        ble.send_command(CMD.SET_TARGET, "20")
        ble.send_command(CMD.SET_MODE, str(int(MODE_TYPE.TAKE_READINGS)))
        ble.send_command(CMD.ENABLE_ROBOT, "")
        await asyncio.sleep(20)
        ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|0.02:|0.0002:|0.0")
        ble.send_command(CMD.SET_MODE, str(int(MODE_TYPE.BRAKE)))
        ble.start_notify(ble.uuid['RX_STRING'], ble_msg_handler)
        CURR_LOG_ARRAY = tof
        print("Retrieving TOF")
        ble.send_command(CMD.RETRIEVE_BUFFER, "TOF")
        while not log_complete:
            await asyncio.sleep(0.5) 
        log_complete = False
        print("TOF retrieved")
        CURR_LOG_ARRAY = pose
        print("Retrieving POSE")
        ble.send_command(CMD.RETRIEVE_BUFFER, "POSE")
        while not log_complete:
            await asyncio.sleep(0.5) 
        log_complete = False
        print("POSE retrieved")
        
        bearings = [20, -20]
        
        tof = np.array(tof).astype('float64')
        pose = np.array(pose).astype('float64')

        ret_tof = np.zeros([18, 2])
        temp = np.zeros([18, 3])

        for i in range(0, 18):
            angle = i * 20
            angle_i = np.argmin(np.abs(pose[:, 3] - angle))
            angle_t = pose[angle_i, 0]
            angle = pose[angle_i, 3]
            tof_reading = tof[np.argmin(np.abs(tof[:, 0] - angle_t)), 1:3] / 1000
            temp[i, :] = [tof_reading[1], tof_reading[0], angle]
            
        ret_tof = temp[:, 0:2]
        print(ret_tof)
            
        return ret_tof, bearings


# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

CURR_LOG_ARRAY = None
tof = []
pose = []
log_complete = False

def ble_msg_handler(sender, msg):
    global log_complete
    msg = msg.decode("utf-8")
    end_regex = r'<END BUFFER [a-z, A-Z]*>'
    if re.search(end_regex, msg) != None:
        print("Buffer read complete.")
        log_complete = True
    msg_regex = r'[A-Za-z]*: -?(\d*.?(?:\d*)?)' # Match Data of the form "[name]: [data]"
    data = re.findall(msg_regex, msg)
    if data == None:
        return
    if len(data) == 0:
        return
    CURR_LOG_ARRAY.append(data)

2023-05-09 10:38:21,817 | INFO     |: Logger demo_notebook.log initialized.
2023-05-09 10:38:22,506 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:83:0c:66:2f:3c
2023-05-09 10:38:25,457 | ERROR    |: [org.bluez.Error.Failed] le-connection-abort-by-local
2023-05-09 10:38:26,460 | WARNING  |: Attempting to connect again...
2023-05-09 10:38:26,462 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:83:0c:66:2f:3c
2023-05-09 10:38:30,141 | ERROR    |: [org.bluez.Error.Failed] le-connection-abort-by-local
2023-05-09 10:38:31,144 | WARNING  |: Attempting to connect again...
2023-05-09 10:38:31,146 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:83:0c:66:2f:3c
2023-05-09 10:38:34,739 | INFO     |: Connected to c0:83:0c:66:2f:3c


/home/michael/Documents/Cornell/FAST-ROBOTS/FAST-ROBOTS-2023/FastRobots-sim-release/notebooks/base_ble.py:87: FutureWarning: This method will be removed future version, pass the callback to the BleakClient constructor instead.
  self.client.set_disconnected_callback(self.disconnect_handler)


In [2]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
log_complete = False

robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2023-05-09 10:38:42,140 | INFO     |:  | Number of observations per grid cell: 18
2023-05-09 10:38:42,143 | INFO     |:  | Precaching Views...
2023-05-09 10:38:42,450 | INFO     |:  | Precaching Time: 0.305 secs
2023-05-09 10:38:42,453 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-05-09 10:38:42,456 | INFO     |: Uniform Belief with each cell value: 3.2150205761316875e-05


# Run an update step of the Bayes Filter

In [ ]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# points = np.array([[-4, -3], [-2, -1], [1, -1], [2, -3], [5, -3], [5, -2], [5, 3], [0, 3], [0, 0]])

# points = points * 0.3048

points = np.array([[0.7,0.2], [0.7, 1.3]])

print(points)

target_index = 1

while True:
    # Get Observation Data by executing a 360 degree rotation motion
    await loc.get_observation_data()

    # Run Update Step
    loc.update_step()
    loc.plot_update_step_data(plot_data=True)
    
    target = points[target_index, :]
    pose_est = np.array(loc.get_current_estimate())
    print("Guess at pose: " + str(pose_est))
    
    print(target)
    print(pose_est[0:2])
    
    if (np.abs(target - pose_est[0:2]) < 0.3).all():
        print("Hit waypoint " + str(target))
        target_index+=1
        target = points[target_index, :]
        print("New target: " + str(target))
        
    
              
    delta = target - pose_est[0:2]
    
    #ble.send_command(CMD.SET_GYRO, str(pose_est[2]))
              
    dist = np.linalg.norm(delta)
    print("Pose estimate: " + str(pose_est[2]))
    angle = np.rad2deg(np.arctan2(delta[1], delta[0])) - pose_est[2] - 5.0
    
    print("Angle to target: " + str(angle))
    print("Distance to target: " + str(dist))
    
    ble.send_command(CMD.SET_GYRO, "0")
    ble.send_command(CMD.SET_TARGET, "-20")
    ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|0.03:|0.0004:|0.0")
    ble.send_command(CMD.SET_MODE, str(int(MODE_TYPE.SEEK_ANGLE)))
    await asyncio.sleep(3)
    ble.send_command(CMD.SET_GYRO, "0")
    ble.send_command(CMD.SET_TARGET, str(angle))
    ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|0.03:|0.0004:|0.0")
    ble.send_command(CMD.SET_MODE, str(int(MODE_TYPE.SEEK_ANGLE)))
    #ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|0.02:|0.0005:|0.0")
    await asyncio.sleep(3)
    ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|0.03:|0.0004:|0.0")
    t = dist * 2000
    print("Moving forward for: " + str(t))
    ble.send_command(CMD.SET_TARGET, str(t))
    ble.send_command(CMD.SET_MODE, str(int(MODE_TYPE.MOVE_FORWARD)))
    await asyncio.sleep(t / 1000.0 + 1)
    ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|0.03:|0.0004:|0.0")
    ble.send_command(CMD.SET_MODE, str(int(MODE_TYPE.BRAKE)))
    await asyncio.sleep(0.5)
        
    

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2023-05-09 10:38:44,224 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-05-09 10:38:44,227 | INFO     |: Uniform Belief with each cell value: 3.2150205761316875e-05
[[0.7 0.2]
 [0.7 1.3]]
starting reading
Retrieving TOF
Buffer read complete.
TOF retrieved
Retrieving POSE
Buffer read complete.
POSE retrieved
[[0.794 0.495]
 [0.668 0.512]
 [0.421 1.145]
 [0.533 1.301]
 [1.284 1.85 ]
 [1.405 0.707]
 [1.887 0.623]
 [0.523 0.543]
 [0.609 0.478]
 [0.51  0.5  ]
 [0.461 0.657]
 [0.491 0.624]
 [0.671 0.506]
 [0.539 0.496]
 [0.464 0.548]
 [0.476 0.78 ]
 [0.565 0.746]
 [0.808 0.62 ]]
2023-05-09 10:39:22,638 | INFO     |: Update Step
2023-05-09 10:39:22,759 | INFO     |:      | Update Time: 0.119 secs
2023-05-09 10:39:22,762 | INFO     |: Bel index     : (14, 12, 34) with prob = 0.4493254
2023-05-09 10:39:22,763 | INFO     |: Bel_bar prob at index = 3.2150205761316875e-05
2023-05-09 10:39:22,765 | INFO     |: Belief        : (0.533, 0.533, -7.500)
Guess at pose: [ 0.5334  0.533

In [2]:
ble.send_command(CMD.SET_PID_GAINS,"ROTATION_RATE:|-0.01:|-0.001:|0.0")
ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|0.01:|0.0002:|0.0")
ble.send_command(CMD.SET_TARGET, "0")
ble.send_command(CMD.SET_GYRO, "0")
ble.send_command(CMD.SET_MODE, str(int(MODE_TYPE.SEEK_ANGLE)))
ble.send_command(CMD.ENABLE_ROBOT, "")

In [17]:
ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|0.001:|0.002:|0.0")

In [6]:
ble.send_command(CMD.DISABLE_ROBOT, "")

In [5]:

ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|0.012:|0.0003:|0.0")
ble.send_command(CMD.SET_TARGET, "180")

In [6]:
ble.send_command(CMD.SET_PID_GAINS,"ROTATION_RATE:|-0.01:|-0.001:|0.0")
ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|0.01:|0.0002:|0.0")
ble.send_command(CMD.SET_TARGET, "3000")
ble.send_command(CMD.SET_GYRO, "0")
ble.send_command(CMD.SET_MODE, str(int(MODE_TYPE.MOVE_FORWARD)))
ble.send_command(CMD.ENABLE_ROBOT, "")


In [3]:
ble.send_command(CMD.DISABLE_ROBOT, "")

Exception: Not connected to a BLE device

In [2]:
ble.send_command(CMD.SET_MODE, str(int(MODE_TYPE.BUG)))
ble.send_command(CMD.ENABLE_ROBOT, "")

2023-05-08 18:22:25,386 | INFO     |: Disconnected from C0:83:0C:66:2F:3C
